# Clustering

In [ ]:
#general packages
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read in data
data = pd.read_csv("/groups/CaiLab/personal/Lex/raw/250113_mb_BSpeg_xtra_potentialTriton/pyfish_tools/output/genebycell/final_1.01.41.4_seed33_heg_svm_p20.0_diff0_fdr5.0/genebycell_1.csv", index_col=0)
#remove rows that doesn't correspond to a gene
data = data[~data.index.str.contains("fake")]

In [ ]:
# Convert to AnnData object
adata = sc.AnnData(data.T)
#make sure cells are obs and genes are vars
adata

In [ ]:
# Filter genes that are expressed in fewer than 1 cells
sc.pp.filter_genes(adata, min_cells=1)
# Filter cells that have fewer than n genes expressed
sc.pp.filter_cells(adata, min_genes=0)
# CPM normalization
sc.pp.normalize_total(adata, target_sum=1e6)
# Log-transform the data
sc.pp.log1p(adata)  
# Use N number of top genes (all since they are all marker genes)
sc.pp.highly_variable_genes(adata, n_top_genes=adata.n_vars, subset=True)
# Z-score normalize and clip any value beyond 10 sigmas
sc.pp.scale(adata, max_value=10)
# Perform PCA to reduce dimensions and keep using arpack for consistent solutions
sc.tl.pca(adata, svd_solver='arpack', n_comps=adata.n_vars-1)

In [ ]:
# Get the explained variance ratio
explained_variance_ratio = adata.uns['pca']['variance_ratio']
# Calculate the cumulative sum of explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
# Find the number of components that account for 90% of the variance
num_pcs_90 = np.argmax(cumulative_variance_ratio >= 0.90) + 1  # Add 1 because indices start at 0
print(f"Number of principal components that account for 90% of the variance: {num_pcs_90}")

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.9, color='#A7C7E7', lw=2)
plt.xlabel('Principal Component', size = 14)
plt.ylabel('Explained Variance Ratio', size = 14)
plt.title('')
# Customize the plot (adding black box edges)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['right'].set_color('black')
# You can also set the thickness of the edges (optional)
plt.gca().spines['top'].set_linewidth(1.5)
plt.gca().spines['bottom'].set_linewidth(1.5)
plt.gca().spines['left'].set_linewidth(1.5)
plt.gca().spines['right'].set_linewidth(1.5)
plt.axvline(num_pcs_90, ls = "--", color = "k")
# Save the plot as an SVG file
#plt.savefig('Variance_Ratio_Plot.svg', format='svg')
plt.show()

In [ ]:
# Generate neighborhood graph. Use only top PCs that gives >90% of variance
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=num_pcs_90)
# Perform UMAP on neighborhood graph
sc.tl.umap(adata, min_dist=0.1, spread=3, random_state=42)
# Perform community based clustering using leiden on neighborhood graph
sc.tl.leiden(adata, resolution=1) 

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

# Automatically adjust palette size based on clusters
num_clusters = adata.obs['leiden'].nunique()
custom_palette = sns.color_palette("tab20", num_clusters)

# Plot UMAP with updated palette
sc.pl.umap(
    adata, 
    color=["leiden"], 
    palette=custom_palette,
    title="",
    edgecolor='black',  
    linewidth=0.2,  # Add edge around the dots
    frameon=False,  
    show=False  
)

ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()

for handle in handles:
    handle.set_edgecolor('black')
    handle.set_linewidth(0.5)

plt.tight_layout()  # Prevents overlapping of plot and legend
plt.show()


# Cell typing

In [ ]:
#load annotations data
annotations = pd.read_csv("/groups/CaiLab/personal/Lex/raw/250113_mb_BSpeg_xtra_potentialTriton/mouse_brain_extra/Xenium_mBrain_v1.1_metadata.csv")

In [ ]:
# Create a boolean mask where "Annotation" contains "L2"
mask = annotations["Annotation"].str.contains("L2 ", case=False, na=False)

# Assign "L2" to the "Annotation" column for these rows
annotations.loc[mask, "Annotation"] = "L2"

# Create a boolean mask where "Annotation" contains "L2"
mask = annotations["Annotation"].str.contains("L3 ", case=False, na=False)

# Assign "L2" to the "Annotation" column for these rows
annotations.loc[mask, "Annotation"] = "L3"

# Create a boolean mask where "Annotation" contains "L2"
mask = annotations["Annotation"].str.contains("L4 ", case=False, na=False)

# Assign "L2" to the "Annotation" column for these rows
annotations.loc[mask, "Annotation"] = "L4 "

# Create a boolean mask where "Annotation" contains "L2"
mask = annotations["Annotation"].str.contains("L5 ", case=False, na=False)

# Assign "L2" to the "Annotation" column for these rows
annotations.loc[mask, "Annotation"] = "L5"

# Create a boolean mask where "Annotation" contains "L2"
mask = annotations["Annotation"].str.contains("L6 ", case=False, na=False)

# Assign "L2" to the "Annotation" column for these rows
annotations.loc[mask, "Annotation"] = "L6"

In [ ]:
annotations.Annotation.unique()

In [ ]:
# Perform differential expression across clusters
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon') 

In [ ]:
#look at dot plot for top 3 genes per cluster
sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden", standard_scale="var", n_genes=3
)

In [ ]:
# Extract the results into a DataFrame
marker_genes = sc.get.rank_genes_groups_df(adata, group=None)

In [ ]:
# Get unique marker genes
unique_marker_genes = marker_genes['names'].unique()

# Filter your annotation DataFrame for these marker genes
annotated_markers = annotations[annotations['Genes'].isin(unique_marker_genes)]

In [ ]:
# Get list of clusters
clusters = adata.obs['leiden'].unique()

# Initialize dictionary
cluster_markers = {}

for cluster in clusters:
    # Extract marker genes for the cluster
    markers_df = marker_genes[marker_genes['group'] == cluster]
    # Select top N marker genes (e.g., top 10)
    top_markers = markers_df.head(5)['names'].tolist()
    cluster_markers[cluster] = top_markers

# Example output
for cluster, genes in cluster_markers.items():
    print(f"Cluster {cluster}: {genes}")

In [ ]:
# Create a dictionary mapping genes to cell types
gene_to_celltype = annotations.set_index('Genes')['Annotation'].to_dict()

# Inspect the mapping
print(list(gene_to_celltype.items())[:10])

In [ ]:
# Get the list of unique cell types
cell_types = annotations['Annotation'].unique()

# Initialize a DataFrame with clusters as rows and cell types as columns
summary_df = pd.DataFrame(0, index=clusters, columns=cell_types)

In [ ]:
for cluster, genes in cluster_markers.items():
    for gene in genes:
        # Check if the gene is in the mapping
        if gene in gene_to_celltype:
            cell_type = gene_to_celltype[gene]
            # If a gene maps to multiple cell types, split and iterate
            if isinstance(cell_type, str) and ',' in cell_type:
                cell_type_list = [ct.strip() for ct in cell_type.split(',')]
            elif isinstance(cell_type, list):
                cell_type_list = cell_type
            else:
                cell_type_list = [cell_type]
            
            for ct in cell_type_list:
                summary_df.at[cluster, ct] += 1

In [ ]:
# Initialize a dictionary to store cluster annotations
cluster_annotations = {}

for cluster in summary_df.index:
    cluster_counts = summary_df.loc[cluster]
    
    # Find the maximum count
    max_count = cluster_counts.max()
    
    if max_count >= 2:
        # Get all cell types with the maximum count
        top_cell_types = cluster_counts[cluster_counts == max_count].index.tolist()
        # Assign the first cell type (you can modify this if needed)
        cluster_annotations[cluster] = '/'.join(top_cell_types)
    elif max_count == 1:
        # Find cell types with count of 1
        single_counts = cluster_counts[cluster_counts == 1]
        if len(single_counts) == 1:
            # Assign the single cell type with count 1
            cluster_annotations[cluster] = single_counts.idxmax()
        else:
            # Multiple cell types with count 1
            cluster_annotations[cluster] = 'Unknown'
    else:
        # No valid counts
        cluster_annotations[cluster] = 'Unknown'

In [ ]:
# Create a mapping Series
cluster_to_celltype = pd.Series(cluster_annotations, name='cell_type')

# Map the annotations to cells based on their cluster
adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_to_celltype)

# Verify the mapping
print(adata.obs[['leiden', 'cell_type']].head())

In [ ]:
# Automatically adjust palette size based on clusters
num_clusters = adata.obs['leiden'].nunique()
custom_palette = sns.color_palette("tab20", num_clusters)

fig, ax = plt.subplots(figsize=(5, 5))  # Adjust figure size as needed

# Plot UMAP on the specified axes
sc.pl.umap(
    adata,
    color='leiden',
    palette=custom_palette,  # or category_colors if supported
    title='',
    legend_fontsize=8,
    legend_fontoutline=0.5,
    edgecolor='black',
    linewidth=0.2,
    show=False,
    ax=ax  # Specify the axes to plot on
)

handles, labels = ax.get_legend_handles_labels()

for handle in handles:
    handle.set_edgecolor('black')
    handle.set_linewidth(0.5)

sns.despine()
plt.tight_layout()

for handle in handles:
    handle.set_edgecolor('black')
    handle.set_linewidth(0.5)

plt.show()


In [ ]:
# Generate a custom palette with a size matching the unique cell types
num_cell_types = adata.obs['cell_type'].nunique()
custom_palette = sns.color_palette("tab20", num_cell_types)

# Create a dictionary to explicitly map each category to a color
category_colors = dict(zip(adata.obs['cell_type'].cat.categories, custom_palette))

fig, ax = plt.subplots(figsize=(6, 6))  # Adjust figure size as needed

# Plot UMAP on the specified axes
sc.pl.umap(
    adata,
    color='cell_type',
    palette=custom_palette,  # or category_colors if supported
    title='',
    legend_fontsize=8,
    legend_fontoutline=0.5,
    edgecolor='black',
    linewidth=0.2,
    show=False,
    ax=ax  # Specify the axes to plot on
)

handles, labels = ax.get_legend_handles_labels()

for handle in handles:
    handle.set_edgecolor('black')
    handle.set_linewidth(0.5)

sns.despine()
plt.tight_layout()
#plt.tight_layout()  # Prevents overlapping of plot and legend
# Save as SVG
output_svg_file = "umap_cell_types.svg"
plt.savefig(output_svg_file, format="svg", dpi=300)  # Specify format and resolution
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(summary_df, annot=True, fmt="d", cmap="YlGnBu")
plt.title('Marker Gene Overlap with Cell Types per Cluster')
plt.xlabel('Cell Types')
plt.ylabel('Clusters')
plt.show()

# Spatial Map

In [ ]:
import tifffile as tf

#path
mask_path = "/groups/CaiLab/personal/Lex/raw/250113_mb_BSpeg_xtra_potentialTriton/pyfish_tools/output/edges_deleted/MMStack_Pos5_z0.tif"
#read mask
mask = tf.imread(mask_path)
# Extract Pos information from filename (assuming consistent naming)
pos = int(mask_path.split("Pos")[1].split("_")[0]) 

#make copy
mask_copy = mask.copy().astype(np.int16)
#grab leiden labels
labels = pd.DataFrame(adata.obs["cell_type"])

In [ ]:
plt.imshow(mask)
plt.show()

In [ ]:
# Parse cell_id and Pos from the index
labels.reset_index(inplace=True)
labels[['cell_id', 'pos']] = labels['index'].str.extract(r'Cell(\d+)\.0_Pos_(\d+)', expand=True).astype(int)
labels = labels[['cell_id', 'pos', 'cell_type']]

# Filter labels for the current Pos
labels = labels[labels['pos'] == pos]

#make cell type dictionary
celltype_def = dict(zip(labels['cell_id'], labels['cell_type']))

In [ ]:
labels

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define a label for the background, if it exists, e.g., 0 for background
background_label = 0

# Extract unique cluster IDs from the mask, excluding the background label
unique_clusters = np.unique(mask_copy)
unique_clusters = unique_clusters[unique_clusters != background_label]  # Remove background label from clusters

# Initialize the mask_colored with a black background
mask_colored = np.zeros((*mask_copy.shape, 3), dtype=float)  # Create an RGB array filled with zeros (black background)

# Apply the color map to the mask
for cluster_id in unique_clusters:
    if cluster_id in celltype_def:
        celltype = celltype_def[int(cluster_id)]
        mask_colored[mask_copy == cluster_id] = category_colors[celltype]
    else:
        # Assign black color to clusters not in celltype_def
        mask_colored[mask_copy == cluster_id] = [0, 0, 0]

# Display the mask with the black background and colored clusters
plt.figure(figsize=(10, 10))
plt.imshow(mask_colored)
plt.axis('off')  # Hide the axes
#plt.savefig("projected_labels_on_masks.svg", bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
## Do for all
from pathlib import Path
output_dir  = Path("/groups/CaiLab/personal/Lex/raw/250113_mb_BSpeg_xtra_potentialTriton/pyfish_tools/output/spatial_mapped_masks")
output_dir.mkdir(parents=True, exist_ok=True)


for pos in range(87):
    try:
        mask_path = f"/groups/CaiLab/personal/Lex/raw/250113_mb_BSpeg_xtra_potentialTriton/pyfish_tools/output/edges_deleted/MMStack_Pos{pos}_z0.tif"
        #read mask
        mask = tf.imread(mask_path)
        # Extract Pos information from filename (assuming consistent naming)
        pos = int(mask_path.split("Pos")[1].split("_")[0]) 
        
        #make copy
        mask_copy = mask.copy().astype(np.int16)
        #grab leiden labels
        labels = pd.DataFrame(adata.obs["cell_type"])
    
        # Parse cell_id and Pos from the index
        labels.reset_index(inplace=True)
        labels[['cell_id', 'pos']] = labels['index'].str.extract(r'Cell(\d+)\.0_Pos_(\d+)', expand=True).astype(int)
        labels = labels[['cell_id', 'pos', 'cell_type']]
        
        # Filter labels for the current Pos
        labels = labels[labels['pos'] == pos]
       
        #make cell type dictionary
        celltype_def = dict(zip(labels['cell_id'], labels['cell_type']))
    
        # Define a label for the background, if it exists, e.g., 0 for background
        background_label = 0
        
        # Extract unique cluster IDs from the mask, excluding the background label
        unique_clusters = np.unique(mask_copy)
        unique_clusters = unique_clusters[unique_clusters != background_label]  # Remove background label from clusters
        
        # Initialize the mask_colored with a black background
        mask_colored = np.zeros((*mask_copy.shape, 3), dtype=float)  # Create an RGB array filled with zeros (black background)
        
        # Apply the color map to the mask
        for cluster_id in unique_clusters:
            if cluster_id in celltype_def:
                celltype = celltype_def[int(cluster_id)]
                mask_colored[mask_copy == cluster_id] = category_colors[celltype]
            else:
                # Assign black color to clusters not in celltype_def
                mask_colored[mask_copy == cluster_id] = [0, 0, 0]
        
        # Display the mask with the black background and colored clusters
        tf.imwrite(str(output_dir / f"MMStack_Pos{pos}.ome.tif"), mask_colored)
    except:
        continue